<a href="https://colab.research.google.com/github/victoreeckh/SDS_assignment/blob/main/SDS_NN_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Smart Distribution Systems (B-KUL-H00P3A)
## Forecasting Competition assignment 
Wout Joris - r0786961\
Victor Eeckhout - r0778787

In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import time
import pickle
import datetime
import datetime as dt
import pytz
import io
from sklearn.model_selection import train_test_split, cross_val_predict, KFold
import os

#%tensorflow_version 1.x

from keras.models import Sequential
from keras.layers.core import Dense
from tensorflow.keras.optimizers import RMSprop, SGD
from keras.callbacks import EarlyStopping

## Data reading and formatting

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
#time values of historical data
start_date = dt.datetime(2017,1,1)
end_date = dt.datetime(2019,12,31)

In [ ]:
# read in Belgian day-ahead electricity prices (DAM prices)
df_day_ahead_price = pd.read_csv(io.BytesIO(uploaded['day-ahead-price.csv']), header=0)
df_day_ahead_price['datetime'] = pd.to_datetime(df_day_ahead_price['datetime'], utc=True)
df_day_ahead_price['datetime'] = df_day_ahead_price['datetime'].dt.tz_localize(None)
df_day_ahead_price.set_index('datetime', inplace=True)

df_day_ahead_price.head()
# print(df_day_ahead_price.dtypes)

In [ ]:
# datacleaning day ahead prices
#empty values
dates = pd.date_range(start=start_date, end=end_date, freq='1H')
# print(dates[:10])
for d in dates:
    try:
        p = df_day_ahead_price.loc[d]
    except KeyError:
        print(d)
#no empty values found
df_day_ahead_price =  df_day_ahead_price.fillna(method='pad')
df_day_ahead_price = df_day_ahead_price.sort_index()
df_day_ahead_price = df_day_ahead_price[start_date:end_date]

frames = {"DAM price": df_day_ahead_price}

In [ ]:
# read in generation import
df_generation_import = pd.read_csv(io.BytesIO(uploaded['generation-import.csv']), header=0, skiprows=[0,2])
df_generation_import = df_generation_import.rename(columns = {'Unnamed: 0':'datetime'})
df_generation_import['datetime'] = pd.to_datetime(df_generation_import['datetime'], utc=True)
df_generation_import['datetime'] = df_generation_import['datetime'].dt.tz_localize(None)
df_generation_import.set_index('datetime', inplace=True)

# print(df_generation_import['datetime'].dtypes)
df_generation_import.head()


In [ ]:
# datacleaning in generation import
Load = 0; 
for col in df_generation_import.columns:
  var_name = "df_generation_import" + "_" + col.replace(" ", "_")
  var_value = df_generation_import[col].fillna(method='pad').fillna(0)
  var_value = var_value[start_date:end_date]
  name_space = globals()
  name_space[var_name] = var_value
  frames[col] = var_value
  Load+=var_value
  # print(var_name)

#Load is sum of generation+import
frames['Load'] = Load
frames['Generation'] = frames['Biomass']+frames['Fossil Gas']+frames['Fossil Oil']+frames['Hydro Pumped Storage']+\
                      +frames['Hydro Run-of-river and poundage']+frames['Nuclear']+frames['Other']+frames['Solar']+frames['Waste']\
                      +frames['Wind Offshore']+ frames['Wind Onshore']
frames['Import'] = frames['NL'] + frames['DE_AT_LU'] + frames['FR'] + frames['GB'] + frames['DE_LU']
frames['Wind'] = frames['Wind Offshore'] + frames['Wind Onshore']

In [ ]:
# read in imbalance price
df_imbalance_price = pd.read_csv(io.BytesIO(uploaded['imbalance-price.csv']), header=0)
df_imbalance_price['datetime'] = pd.to_datetime(df_imbalance_price['datetime'], utc=True)
df_imbalance_price['datetime'] = df_imbalance_price['datetime'].dt.tz_localize(None)
df_imbalance_price.set_index('datetime', inplace=True)

df_imbalance_price_long = df_imbalance_price["Long"]
df_imbalance_price_short = df_imbalance_price["Short"]

# print(df_imbalance_price.dtypes)
df_imbalance_price.head()

In [ ]:
# datacleaning imbalance prices
#empty values
for d in dates:
    try:
        p = df_imbalance_price_long.loc[d]
    except KeyError:
        print(d)
#no empty values found
df_imbalance_price_long =  df_imbalance_price_long.fillna(method='pad')
df_imbalance_price_long = df_imbalance_price_long.sort_index()
df_imbalance_price_long = df_imbalance_price_long[start_date:end_date]
frames["imbalance price long"] = df_imbalance_price_long

for d in dates:
    try:
        p = df_imbalance_price_short.loc[d]
    except KeyError:
        print(d)
#no empty values found
df_imbalance_price_short =  df_imbalance_price_short.fillna(method='pad')
df_imbalance_price_short = df_imbalance_price_short.sort_index()
df_imbalance_price_short = df_imbalance_price_short[start_date:end_date]
frames["imbalance price short"] = df_imbalance_price_short

In [ ]:
#read in solar values
df_solar_and_wind_forecast = pd.read_csv(io.BytesIO(uploaded['wind-and-solar.csv']), header=0)
df_solar_and_wind_forecast['datetime'] = pd.to_datetime(df_solar_and_wind_forecast['datetime'], utc=True)
df_solar_and_wind_forecast['datetime'] = df_solar_and_wind_forecast['datetime'].dt.tz_localize(None)
df_solar_and_wind_forecast.set_index('datetime', inplace=True)

df_solar_forecast = df_solar_and_wind_forecast['Solar']
df_wind_offshore_forecast = df_solar_and_wind_forecast['Wind Offshore']
df_wind_onshore_forecast = df_solar_and_wind_forecast['Wind Onshore']

df_solar_and_wind_forecast.head()
# df_solar.head()
# df_wind_onshore.head()

In [ ]:
# datacleaning wind and solar
df_solar_forecast =  df_solar_forecast.fillna(method='pad')
df_solar_forecast = df_solar_forecast[start_date:end_date]
frames["solar forecast"] = df_solar_forecast
df_wind_offshore_forecast =  df_wind_offshore_forecast.fillna(method='pad')
df_wind_offshore_forecast = df_wind_offshore_forecast[start_date:end_date]
frames["wind offshore forecast"] = df_wind_offshore_forecast
df_wind_onshore_forecast =  df_wind_onshore_forecast.fillna(method='pad')
df_wind_onshore_forecast = df_wind_onshore_forecast[start_date:end_date]
frames["wind onshore forecast"] = df_wind_onshore_forecast
frames["wind forecast"] = df_wind_onshore_forecast + df_wind_offshore_forecast

In [ ]:
# Fixing granularities
d = {}
for key,frame in frames.items():
  # print(type(frame))
  frames[key] = frame.resample('15T').pad()
  d[key] = frames[key].values
  # print(frame)
  # print(len(frame))
d["DAM price"] = d["DAM price"].flatten()


In [ ]:
# creating one dataframe

data = pd.DataFrame(index=frames["DAM price"].index, data=d)
data.head(10)

In [ ]:
# removing outliers
#(only for prices)

# print(frames.keys())

for key in {"DAM price", "imbalance price long", "imbalance price short"}:
  frame = data[key]
  mean = frame.mean()
  # print(mean)
  std = frame.std()
  # print(std)
  if key == "DAM price":
    n_std = 5
  else:
    n_std = 10

  data[key][(frame >= mean + n_std*std)] = mean + n_std*std 
  data[key][(frame <= mean - n_std*std)] = mean + n_std*std 

#accuracy function
def get_accuracy(x, y):
    return np.mean(np.abs(x - y))/np.mean(x)


In [ ]:
# some figures
plt.figure()
plt.subplot(311)
plt.plot(data["DAM price"], label="DAM price")
plt.legend(frameon=False)
plt.subplot(312)
plt.plot(data["imbalance price long"], label="imbalance price long")
plt.legend(frameon=False)
plt.subplot(313)
plt.plot(data["imbalance price short"], label="imbalance price short")
plt.legend(frameon=False)
plt.show()

## Training Dataset

### Autocorrelation
Biggest correlation with previous day, followed by the day before and the same day in the previous week is also substantially autocorrelated. We will use these features to construct the training data.

In [ ]:
#Autocorellation
lags = np.arange(1, 96*10)
acors = []
for lag in lags:
    acors.append(data['DAM price'].autocorr(lag))
plt.figure()
plt.plot(lags/4/24.0, acors)
plt.xlabel('Time lag in days')
plt.grid(True)

In [ ]:
n_hours = 24
#one day ahead
start_X_1_d_ahead = datetime.datetime(2017, 1, 7, 0, 0)
end_X_1_d_ahead = datetime.datetime(2018, 12, 29, 23, 45)
X_1_d_ahead = data['DAM price'][start_X_1_d_ahead:end_X_1_d_ahead].resample('1H').mean().values.reshape(-1, n_hours)

#two days ahead
start_X_2_d_ahead = datetime.datetime(2017, 1, 6, 0, 0)
end_X_2_d_ahead = datetime.datetime(2018, 12, 28, 23, 45)
X_2_d_ahead = data['DAM price'][start_X_2_d_ahead:end_X_2_d_ahead].resample('1H').mean().values.reshape(-1, n_hours)

#a week ahead
start_X_7_d_ahead = datetime.datetime(2017, 1, 1, 0, 0)
end_X_7_d_ahead = datetime.datetime(2018, 12, 23, 23, 45)
X_7_d_ahead = data['DAM price'][start_X_7_d_ahead:end_X_7_d_ahead].resample('1H').mean().values.reshape(-1, n_hours)

start_Y = datetime.datetime(2017, 1, 8, 0, 0)
end_Y = datetime.datetime(2018, 12, 30, 23, 45)

Y = data['DAM price'][start_Y:end_Y].resample('1H').mean().values.reshape(-1, n_hours)

# solar load data and forecast
S_load = data['Solar'][start_Y:end_Y].resample('1H').mean().values.reshape(-1, n_hours)
S_forecast = data['solar forecast'][start_Y:end_Y].resample('1H').mean().values.reshape(-1, n_hours)

# wind (off- + onshore) load data and forecast
W_load = data['Wind'][start_Y:end_Y].resample('1H').mean().values.reshape(-1, n_hours)
W_forecast = data['wind forecast'][start_Y:end_Y].resample('1H').mean().values.reshape(-1, n_hours)

#Total load
L = data['Load'][start_Y:end_Y].resample('1H').mean().values.reshape(-1, n_hours)

#Imbalance price long
I_long = data['imbalance price long'][start_Y:end_Y].resample('1H').mean().values.reshape(-1, n_hours)

#Imbalance price short
I_short = data['imbalance price short'][start_Y:end_Y].resample('1H').mean().values.reshape(-1, n_hours)

#Import
Im = data['Import'][start_Y:end_Y].resample('1H').mean().values.reshape(-1, n_hours)

#Generation
G = data['Generation'][start_Y:end_Y].resample('1H').mean().values.reshape(-1, n_hours)

##Construct training matrix
rows = X_1_d_ahead.shape[0]
col = X_1_d_ahead.shape[1]

#CHANGE FEATURES HERE
use_X_1_d_ahead = 1
use_X_2_d_ahead = 0
use_X_7_d_ahead = 1
use_S_forecast = 1
use_W_forecast = 0
use_S_Load = 0
use_W_Load = 0
use_L = 0
use_I_long = 1
use_I_short = 1
use_Import = 0
use_G = 0

n = use_X_1_d_ahead + use_X_2_d_ahead + use_X_7_d_ahead + use_S_forecast\
    + use_W_forecast + use_S_Load + use_W_Load + use_L+use_I_long+use_I_short\
    +use_Import + use_G
T = np.zeros((rows,n*col))

for i in range(0,rows):
  j = 0
  while j!=n:
    if use_X_1_d_ahead:
      T[i,j*col:(j+1)*col] = X_1_d_ahead[i,0:col]
      j+=1
    if use_X_2_d_ahead:
      T[i,j*col:(j+1)*col] = X_2_d_ahead[i,0:col]
      j+=1
    if use_X_7_d_ahead:
      T[i,j*col:(j+1)*col] = X_7_d_ahead[i,0:col]
      j+=1
    if use_S_forecast:
      T[i,j*col:(j+1)*col] = S_forecast[i,0:col]
      j+=1
    if use_W_forecast:
      T[i,j*col:(j+1)*col] = W_forecast[i,0:col]
      j+=1
    if use_S_Load:
      T[i,j*col:(j+1)*col] = S_load[i,0:col]
      j+=1
    if use_W_Load:
      T[i,j*col:(j+1)*col] = W_load[i,0:col]
      j+=1
    if use_L:
      T[i,j*col:(j+1)*col] = L[i,0:col]
      j+=1
    if use_I_long:
      T[i,j*col:(j+1)*col] = I_long[i,0:col]
      j+=1
    if use_I_short:
      T[i,j*col:(j+1)*col] = I_short[i,0:col]
      j+=1
    if use_Import:
      T[i,j*col:(j+1)*col] = Im[i,0:col]
      j+=1
    if use_G:
      T[i,j*col:(j+1)*col] = G[i,0:col]
      j+=1


# for i in range(0,rows):
#     T[i,0:col] = X_1_d_ahead[i,0:col]
#     T[i,col:2*col] = X_2_d_ahead[i,0:col]
#     T[i,2*col:3*col] = X_7_d_ahead[i,0:col]
#     T[i,3*col:4*col] = S_forecast[i,0:col]
#     T[i,4*col:5*col] = W_forecast[i,0:col]
#     T[i,5*col:6*col] = L[i,0:col]
print('shape of T ' + str(T.shape))
Neurons_l_1 = T.shape[1]

## Validation dataset

In [ ]:
n_hours = 24
#one day ahead
start_X_1_d_ahead = datetime.datetime(2019, 1, 7, 0, 0)
end_X_1_d_ahead = datetime.datetime(2019, 12, 29, 23, 45)
X_1_d_ahead = data['DAM price'][start_X_1_d_ahead:end_X_1_d_ahead].resample('1H').mean().values.reshape(-1, n_hours)

#two days ahead
start_X_2_d_ahead = datetime.datetime(2019, 1, 6, 0, 0)
end_X_2_d_ahead = datetime.datetime(2019, 12, 28, 23, 45)
X_2_d_ahead = data['DAM price'][start_X_2_d_ahead:end_X_2_d_ahead].resample('1H').mean().values.reshape(-1, n_hours)

#a week ahead
start_X_7_d_ahead = datetime.datetime(2019, 1, 1, 0, 0)
end_X_7_d_ahead = datetime.datetime(2019, 12, 23, 23, 45)
X_7_d_ahead = data['DAM price'][start_X_7_d_ahead:end_X_7_d_ahead].resample('1H').mean().values.reshape(-1, n_hours)

start_Y = datetime.datetime(2019, 1, 8, 0, 0)
end_Y = datetime.datetime(2019, 12, 30, 23, 45)

Y_val = data['DAM price'][start_Y:end_Y].resample('1H').mean().values.reshape(-1, n_hours)

# solar load data and forecast
S_load = data['Solar'][start_Y:end_Y].resample('1H').mean().values.reshape(-1, n_hours)
S_forecast = data['solar forecast'][start_Y:end_Y].resample('1H').mean().values.reshape(-1, n_hours)

# wind (off- + onshore) load data and forecast
W_load = data['Wind'][start_Y:end_Y].resample('1H').mean().values.reshape(-1, n_hours)
W_forecast = data['wind forecast'][start_Y:end_Y].resample('1H').mean().values.reshape(-1, n_hours)

#Total load
L = data['Load'][start_Y:end_Y].resample('1H').mean().values.reshape(-1, n_hours)

##Construct training matrix
rows = X_1_d_ahead.shape[0]
col = X_1_d_ahead.shape[1]

T_val = np.zeros((rows,n*col))

for i in range(0,rows):
  j = 0
  while j!=n:
    if use_X_1_d_ahead:
      T_val[i,j*col:(j+1)*col] = X_1_d_ahead[i,0:col]
      j+=1
    if use_X_2_d_ahead:
      T_val[i,j*col:(j+1)*col] = X_2_d_ahead[i,0:col]
      j+=1
    if use_X_7_d_ahead:
      T_val[i,j*col:(j+1)*col] = X_7_d_ahead[i,0:col]
      j+=1
    if use_S_forecast:
      T_val[i,j*col:(j+1)*col] = S_forecast[i,0:col]
      j+=1
    if use_W_forecast:
      T_val[i,j*col:(j+1)*col] = W_forecast[i,0:col]
      j+=1
    if use_S_Load:
      T_val[i,j*col:(j+1)*col] = S_load[i,0:col]
      j+=1
    if use_W_Load:
      T_val[i,j*col:(j+1)*col] = W_load[i,0:col]
      j+=1
    if use_L:
      T_val[i,j*col:(j+1)*col] = L[i,0:col]
      j+=1
    if use_I_long:
      T[i,j*col:(j+1)*col] = I_long[i,0:col]
      j+=1
    if use_I_short:
      T[i,j*col:(j+1)*col] = I_short[i,0:col]
      j+=1
    if use_Import:
      T[i,j*col:(j+1)*col] = Im[i,0:col]
      j+=1
    if use_G:
      T[i,j*col:(j+1)*col] = G[i,0:col]
      j+=1


# for i in range(0,rows):
#     T_val[i,0:col] = X_1_d_ahead[i,0:col]
#     T_val[i,col:2*col] = X_2_d_ahead[i,0:col]
#     T_val[i,2*col:3*col] = X_7_d_ahead[i,0:col]
#     T_val[i,3*col:4*col] = S_forecast[i,0:col]
#     T_val[i,4*col:5*col] = W_forecast[i,0:col]
#     T_val[i,5*col:6*col] = L[i,0:col]
print('shape of T_val ' + str(T_val.shape))

## Training NN

In [ ]:
neurons = [Neurons_l_1,120,24]
activation_functions = ['relu','relu','linear']

model = Sequential()
model.add(Dense(neurons[0], input_dim=T.shape[1], activation=activation_functions[0]))
model.add(Dense(neurons[1], activation=activation_functions[1]))
model.add(Dense(neurons[2], activation=activation_functions[2]))

rprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-6)
model.compile(loss='mean_squared_error', optimizer=rprop)

output_training = model.fit(T, Y, epochs=700, batch_size=32, verbose=0, validation_data=(T_val,Y_val))
mse = output_training.history['loss'][-1]
print('- mse is %.4f' % mse + ' @ ' + str(len(output_training.history['loss'])))

### Early stopping

In [ ]:
# Weights
ww = model.get_weights()

#Running mean function
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

mean_window= 20
plt.figure()
train_loss = plt.plot(running_mean(x=output_training.history['loss'], N=mean_window), label='training error')
val_loss = plt.plot(running_mean(x=output_training.history['val_loss'], N=mean_window), color='red', label='validation error')
plt.xlim([0, 500])
plt.ylim([0, 500])
plt.legend()
plt.show()

## Prediction

In [ ]:
#on the basis of training set
predict_nn = model.predict(T)

print(get_accuracy(Y.flatten(), predict_nn.flatten()))

# Plots
plt.figure()
plt.plot(Y[:10,:].flatten(), color='blue', label='actual price')
plt.plot(predict_nn[:10,:].flatten(), color='red', label='forecast NN')
plt.legend(frameon=False)
plt.show()

In [ ]:
#on the basis of validation set

Y_pred = model.predict(T_val)
print('input_features ' + str(Y_val.shape))

def mean_squared_error(y1, y2):
    return np.mean(np.power(y1-y2, 2), axis=0)

mse = mean_squared_error(np.reshape(Y_val, (8568, 1)), np.reshape(Y_pred, (8568, 1)))
print(mse)

plt.figure()
plt.plot(Y_val[:10,:].flatten(), color='blue', label='actual price')
plt.plot(Y_pred[:10,:].flatten(), color='red', label='forecast NN')
plt.legend(frameon=False)
plt.show()

In [ ]:
from google.colab import files
uploaded = files.upload()

# Read in week 1

In [ ]:
## First week

#input price data
#granularity already in hours

# df_day_ahead_price = pd.read_csv(io.BytesIO(uploaded['day-ahead-price_week1.csv']), header=0)
# df_day_ahead_price.head()
df_day_ahead_price = np.array(pd.read_csv('day-ahead-price_week1.csv', header=0))
# print(len(df_day_ahead_price))
# print(df_day_ahead_price)

In [ ]:
#input forecasted solar and wind
# granularity already in hours
df_solar_and_wind_forecast = pd.read_csv(io.BytesIO(uploaded['wind-and-solar_week1.csv']), header=0)
df_solar_and_wind_forecast.head()
df_solar_forecast = df_solar_and_wind_forecast['Solar']
df_wind_forecast = df_solar_and_wind_forecast['Wind Offshore'] + df_solar_and_wind_forecast['Wind Onshore']
# df_solar_forecast.head()
# df_wind_forecast.head()


In [ ]:
#input Imbalance prices
#granularity still 15'
df_imbalance_price = pd.read_csv(io.BytesIO(uploaded['imbalance-price_week1.csv']), header=0)
df_imbalance_price_long = df_imbalance_price['Long']
df_imbalance_price_short = df_imbalance_price['Short']
# df_imbalance_price.head()
# df_imbalance_price_long.head()
# df_imbalance_price_short.head()

# 15' --> 1h
# by interpolation
i = int(len(df_imbalance_price_long)/4)
imbalance_long = np.zeros((i))

for r in range(0,i-1):
   imbalance_long[r] = (df_imbalance_price_long[r*4]+df_imbalance_price_long[r*4+1]+df_imbalance_price_long[r*4+2]+df_imbalance_price_long[r*4+3])/4
print(imbalance_long.shape)

imbalance_short = np.zeros((i))

for r in range(0,i-1):
   imbalance_short[r] = (df_imbalance_price_short[r*4]+df_imbalance_price_short[r*4+1]+df_imbalance_price_short[r*4+2]+df_imbalance_price_short[r*4+3])/4
print(imbalance_short.shape)

In [ ]:
#Combine code in a matrix
#Depends on chosen features

D = np.zeros((1,120))
p = len(df_solar_forecast) #7d of data available
q = len(df_day_ahead_price) #only 6d of data available

D[0,0:24] = df_day_ahead_price[q-24:q,0]
D[0,24:2*24] = df_day_ahead_price[0:24,0]
D[0,2*24:3*24] = df_solar_forecast[p-24:p]
D[0,3*24:4*24] = imbalance_long[p-24:p]
D[0,4*24:5*24] = imbalance_short[p-24:p]

A1 = model.predict(D)

plt.figure()
plt.plot(A1.flatten(), color='red', label='forecast NN')
plt.legend(frameon=False)
plt.show()

# Read in week 2

In [ ]:
## Second week

#input price data
#granularity already in hours

# df_day_ahead_price = pd.read_csv(io.BytesIO(uploaded['day-ahead-price_week2.csv']), header=0)
# df_day_ahead_price.head()
df_day_ahead_price2 = np.array(pd.read_csv('day-ahead-price_week2.csv', header=0))
# print(len(df_day_ahead_price))
# print(df_day_ahead_price)

In [ ]:
#input forecasted solar and wind
# granularity already in hours
df_solar_and_wind_forecast2 = pd.read_csv(io.BytesIO(uploaded['wind-and-solar_week2.csv']), header=0)
df_solar_and_wind_forecast2.head()
df_solar_forecast2 = df_solar_and_wind_forecast2['Solar']
df_wind_forecast2 = df_solar_and_wind_forecast2['Wind Offshore'] + df_solar_and_wind_forecast2['Wind Onshore']
# df_solar_forecast.head()
# df_wind_forecast.head()

In [ ]:
#input Imbalance prices
#granularity still 15'
df_imbalance_price2 = pd.read_csv(io.BytesIO(uploaded['imbalance-price_week2.csv']), header=0)
df_imbalance_price_long2 = df_imbalance_price2['Long']
df_imbalance_price_short2 = df_imbalance_price2['Short']
# df_imbalance_price.head()
# df_imbalance_price_long.head()
# df_imbalance_price_short.head()

# 15' --> 1h
# by interpolation
i = int(len(df_imbalance_price_long2)/4)
imbalance_long2 = np.zeros((i))

for r in range(0,i-1):
   imbalance_long2[r] = (df_imbalance_price_long2[r*4]+df_imbalance_price_long2[r*4+1]+df_imbalance_price_long2[r*4+2]+df_imbalance_price_long2[r*4+3])/4
print(imbalance_long2.shape)

imbalance_short2 = np.zeros((i))

for r in range(0,i-1):
   imbalance_short2[r] = (df_imbalance_price_short2[r*4]+df_imbalance_price_short2[r*4+1]+df_imbalance_price_short2[r*4+2]+df_imbalance_price_short2[r*4+3])/4
print(imbalance_short2.shape)

In [ ]:
#Combine code in a matrix
#Depends on chosen features

D = np.zeros((1,120))
p = len(df_solar_forecast) #7d of data available
q = len(df_day_ahead_price) #only 6d of data available

D[0,0:24] = df_day_ahead_price2[q-24:q,0]
D[0,24:2*24] = df_day_ahead_price2[0:24,0]
D[0,2*24:3*24] = df_solar_forecast2[p-24:p]
D[0,3*24:4*24] = imbalance_long2[p-24:p]
D[0,4*24:5*24] = imbalance_short2[p-24:p]

A2 = model.predict(D)

plt.figure()
plt.plot(A2.flatten(), color='red', label='forecast NN')
plt.legend(frameon=False)
plt.show()

# Read in week 3

In [ ]:
## Third week

#input price data
#granularity already in hours

# df_day_ahead_price = pd.read_csv(io.BytesIO(uploaded['day-ahead-price_week3.csv']), header=0)
# df_day_ahead_price.head()
df_day_ahead_price3 = np.array(pd.read_csv('day-ahead-price_week3.csv', header=0))
# print(len(df_day_ahead_price))
# print(df_day_ahead_price)

In [ ]:
#input forecasted solar and wind
# granularity already in hours
df_solar_and_wind_forecast3 = pd.read_csv(io.BytesIO(uploaded['wind-and-solar_week3.csv']), header=0)
df_solar_and_wind_forecast3.head()
df_solar_forecast3 = df_solar_and_wind_forecast3['Solar']
df_wind_forecast3 = df_solar_and_wind_forecast3['Wind Offshore'] + df_solar_and_wind_forecast3['Wind Onshore']
# df_solar_forecast.head()
# df_wind_forecast.head()

In [ ]:
#input Imbalance prices
#granularity still 15'
df_imbalance_price3 = pd.read_csv(io.BytesIO(uploaded['imbalance-price_week3.csv']), header=0)
df_imbalance_price_long3 = df_imbalance_price3['Long']
df_imbalance_price_short3 = df_imbalance_price3['Short']
# df_imbalance_price.head()
# df_imbalance_price_long.head()
# df_imbalance_price_short.head()

# 15' --> 1h
# by interpolation
i = int(len(df_imbalance_price_long3)/4)
imbalance_long3 = np.zeros((i))

for r in range(0,i-1):
   imbalance_long3[r] = (df_imbalance_price_long3[r*4]+df_imbalance_price_long3[r*4+1]+df_imbalance_price_long3[r*4+2]+df_imbalance_price_long3[r*4+3])/4
print(imbalance_long3.shape)

imbalance_short3 = np.zeros((i))

for r in range(0,i-1):
   imbalance_short3[r] = (df_imbalance_price_short3[r*4]+df_imbalance_price_short3[r*4+1]+df_imbalance_price_short3[r*4+2]+df_imbalance_price_short3[r*4+3])/4
print(imbalance_short3.shape)

In [ ]:
#Combine code in a matrix
#Depends on chosen features

D = np.zeros((1,120))
p = len(df_solar_forecast) #7d of data available
q = len(df_day_ahead_price) #only 6d of data available

D[0,0:24] = df_day_ahead_price3[q-24:q,0]
D[0,24:2*24] = df_day_ahead_price3[0:24,0]
D[0,2*24:3*24] = df_solar_forecast3[p-24:p]
D[0,3*24:4*24] = imbalance_long3[p-24:p]
D[0,4*24:5*24] = imbalance_short3[p-24:p]

A3 = model.predict(D)

plt.figure()
plt.plot(A3.flatten(), color='red', label='forecast NN')
plt.legend(frameon=False)
plt.show()

# Putting predictions in file

In [ ]:
P = np.zeros((72))

P[0:24] = A1
P[24:48] = A2
P[48:72] = A3

P = np.transpose(P)
P_data = DataFrame(P)

P_data.to_csv('predictions.csv', header=False)
# P_data.head(72)